# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Roger Amorós Sirera  <br>
Url: https://github.com/RogerAmoros13/algoritmos_de_optimizacion <br>
Problema: 
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>


Descripción del problema:(copiar enunciado)

....

(*) La respuesta es obligatoria







### Problema 1

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible

##### Pregunta 1:

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>


¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Este problema se puede resumir en de que manera ordenamos las tomas para que tengamos que haceer desplazarse un mínimo de veces a cada actor. Esto tiene, a priori, tantas posibilidades como maneras de ordenar las *n* tomas hay, es decir, *n!*.

No obstante, podemos matizar la respuesta y hilar más fino, ya que podemos observar que hay respuestas que son equivalentes y por tanto descartables. Supongamos un problema reducido, con 4 tomas, pudiendo realizar un máximo de 2 por día. Según la respuesta a priori, tendríamos las siguientes soluciones basadas en las permutaciones:

[1, 2, 3, 4], [1, 2, 4, 3], [1, 3, 2, 4], [1, 3, 4, 2], [1, 4, 2, 3], [1, 4, 3, 2], [2, 1, 3, 4], [2, 1, 4, 3], [2, 3, 1, 4], [2, 3, 4, 1], [2, 4, 1, 3], [2, 4, 3, 1], [3, 1, 2, 4], [3, 1, 4, 2], [3, 2, 1, 4], [3, 2, 4, 1], [3, 4, 1, 2], [3, 4, 2, 1], [4, 1, 2, 3], [4, 1, 3, 2], [4, 2, 1, 3], [4, 2, 3, 1], [4, 3, 1, 2], [4, 3, 2, 1]

Tenemos 24 soluciones pero muchas de ellas son equivalentes, por ejemplo *[1, 2, 3, 4]* y *[1, 2, 4, 3]* son la misma, ya que los mismos actores tienen que ir los mismos días dado que el orden de las escenas dentro de un día no es importante. Esto obviamente nos reduce sustancialmente el número de soluciones y al mismo tiempo vuelve más difícil de calcular cuantas combinaciones hay.

Después de pensarlo bastante, realizar varias pruebas con combinatorios, no supe deducir la formula general por mi mismo. Finalmente encontre un paper sobre el tema y pude extraer conclusiones sobre el número de posibilidades que tenemos para el problema con *n* tomas totales y *m* tomas diarias máximas. El paper en cuestión se encuentra al final en la bibliografía.

$$
\phi_{n, m} = \frac{n!}{m!^{c}c!}
$$

donde $$c = \frac{n}{m}$$

Es obvio que $\phi_{n, m} < n!$, por lo que teniendo esto en cuenta, a la hora de calcular todas las posibilidades a fuerza aprovechando esto el resultado sera más optimo. Aun así, nuestro problema cuenta con *1.142395139657772e+16* posibilidades.

In [1]:
# En esta celda tenemos definido el factorial y la función que calcula el total
# de posibilidades.

def fact(N):
    if N == 1:
        return 1
    return N * fact(N-1)

def total_posibilities(n, m):
    n_fact = fact(n)
    m_fact = fact(m)
    n_m = int(n/m)
    c = fact(n_m)
    return int(n_fact / (m_fact ** n_m * c))

TOTAL_POSIBILITIES = fact(30)
EFFICIENT_POSIBILITIES = total_posibilities(30, 6)
print(TOTAL_POSIBILITIES, EFFICIENT_POSIBILITIES)
print(TOTAL_POSIBILITIES < EFFICIENT_POSIBILITIES)

265252859812191058636308480000000 11423951396577720
False


##### Pregunta 2:

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)


Respuesta



Los datos de entrada se guardarán en una matriz 10x30, es decir una lista de listas con las tomas de cada actor. Esto es, insertar la matriz de restricciones traspuesta. En la siguiente celda veremos el código y un *print* para ver el formato.

Como mención, también pensé insertar los datos en una matriz normal 30x10 como vienen inicialmente o mediante un diccionario cuyas llaves eran las tomas y los valores una lista con los actores. Esto será preciso más adelante, pero ya haremos los ajustes pertinentes entonces.

Por otro lado, la solución vendra dada por una lista de listas, donde en cada lista estarán las tomas de ese día, por ejemplo:

*[ <br>
    ["Toma 1", "Toma 5", "Toma 6", "Toma 15", "Toma 20", "Toma 21"], <br>
    ["Toma 4", "Toma 2", "Toma 3", "Toma 16", "Toma 28", "Toma 27"], <br>
    ..., <br>
    ["Toma 7", "Toma 8", "Toma 9", "Toma 17", "Toma 23", "Toma 26"], <br>
]*

In [273]:
# Aquí mostramos el código para la recepción de los datos del problema, en un csv ya limpio,
# es decir, 30 filas y 11 columnas (numero de toma, actores*10...).

import csv

NUM_TOMAS = 30 # Prefijamos el número de tomas
NUM_ACTORS = 10 # Prefijamos el número de actores
MAX_TOMAS = 6 # Prefijamos el número de tomas máximo
TOMAS_LIST = [i for i in range(NUM_TOMAS)]
TOMAS = [[] for _ in range(NUM_ACTORS-1)] # Datos de las tomas

with open("datos_escenas.csv", "r") as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        for i in range(1, NUM_ACTORS):
            # Cada fila la añadimos a la columna.
            TOMAS[i-1].append(int(row[i]))

print(*TOMAS, sep="\n")

[1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [275]:
# Vamos a presentar la función encargada de comprobar que la solución es correcta
# y el coste que tiene. Teniendo en cuenta que un algoritmo bien implementado solo
# debería producir soluciones validas, crearemos otra función sin estas comprobaciones.

# Como hemos anticipado, la solución será una matriz diasxtomas. Si suponemos que las 
# mejores soluciones son de 5 días, es decir, seis tomas por día podríamos usar simplemente 
# una lista con el orden de las tomas.

def compute_cost_and_test(solution):
    """
    solution <list>: Lista de listas.
    return: Coste de la solución. Si devuelve 0, respuesta no 
            válida. (Cualquier solución tendrá un coste mayor que 0)
    """
    
    actor_moves = 0
    takes_done = []
    for day in solution:
        if len(day) > 6: # Solución nula si has más de 6 tomas en un día
            return 0
        actors = set()
        for take in day:
            if take in takes_done: # Solución nula si se duplican tomas
                return 0
            takes_done.append(take)
            for i in range(len(TOMAS)):
                if TOMAS[i][take]:
                    actors.add(i)
        actor_moves += len(actors)
    if len(takes_done) < len(TOMAS[0]): # Solución nula si no estan todas las tomas
        return 0
    return actor_moves

def compute_cost(solution):
    """
    solution <list>: Lista de listas.
    return: Coste de la solución. Si devuelve 0, respuesta no 
            válida. (Cualquier solución tendrá un coste mayor que 0)
    """
    
    actor_moves = 0
    for day in solution: # iteración sobre n/m items
        actors = set()
        for take in day: # iteración sobre m items
            for i in range(len(TOMAS)): # iteración sobre n items
                if TOMAS[i][take]:
                    actors.add(i) # Operación básica
        actor_moves += len(actors)
    return actor_moves

Aprovechamos para indicar que el algoritmo *compute_cost* tiene un coste de 
$$
    \frac{n}{m}mn = n^2 \to O(n^2)
$$

In [277]:
# Para ver como funciona vamos a realizar una serie de tests con una solución válida y
# tres erróneas.

print("-"*35 + " Tests para la función compute_cost_and_test " + "-"*35)

sol = [[i for i in range(MAX_TOMAS*(j-1), MAX_TOMAS*j)] for j in range(1, MAX_TOMAS)]

print("\nSolución trivial")
print(sol)
print(compute_cost_and_test(sol))

print("\nSolución errónea: Día con tomas de más")
sol[0].append(6)
print(sol)
print(compute_cost_and_test(sol))
sol[0].pop() # Restauramos la solución inicial

print("\nSolución erronea: Toma repetida")
sol[4].pop() # Sustituimos la última toma por la 0
sol[4].append(0)
print(sol)
print(compute_cost_and_test(sol))

print("\nSolución erronea: Falta una o más tomas")
sol[4].pop()
print(sol)
print(compute_cost_and_test(sol))

----------------------------------- Tests para la función compute_cost_and_test -----------------------------------

Solución trivial
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29]]
36

Solución errónea: Día con tomas de más
[[0, 1, 2, 3, 4, 5, 6], [6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29]]
0

Solución erronea: Toma repetida
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 0]]
0

Solución erronea: Falta una o más tomas
[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17], [18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28]]
0


In [280]:
# Finalmente, para no repetir código creamos una funcion que imprime un texto
# con la solución.

def print_sol(solution, cost):
    if cost:
        print("La mejor solución es: ")
    else:
        print("Solución erronea!")
    print(*solution, sep="\n")
    if cost:
        print(f"Tiene un coste de {cost} con un total de {len(solution)} días de grabación")
        
print_sol(sol, compute_cost(sol)) # Solución anterior incorrecta

print("\n")
sol[-1].append(29)
print_sol(sol, compute_cost(sol)) # Solución correcta

La mejor solución es: 
[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]
[12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29, 29]
Tiene un coste de 36 con un total de 5 días de grabación


La mejor solución es: 
[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]
[12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29, 29, 29]
Tiene un coste de 36 con un total de 5 días de grabación


##### Pregunta 3:

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Nuestro objetivo es minimizar la cantidad de desplazamientos diarios que se han de realizar para grabar las tomas. En lenguage común sería tratar de *"compactar los actores para que vengan los menos días posibles"*. De una manera más matemática, la función a minimizar se escribiría como

$$
f(x) = \sum_i^{n/m} |D_i|
$$
donde $|D_i|$ es el cardinal del grupo de actores que han participado el día $i$.

##### Pregunta 4:

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [283]:
import time

# Aquí tenemos una función que recibe una lista de números ordenada y devuelve todas sus permutaciones.
# Se puede permutar una lista de manera más compacta recursivamente. Pero debido a que hay 30! permutaciones
# directamente ni tratamos de utilizarla, ya que tendríamos que ir guardando todos los resultados para
# usarlos al final. Es por esto que definimos una función no-recursiva y generatriz, para devolver resultado
# a resultado sin tener que guardar en memoria.

def get_permutation(nums=None, i=0):
    if i == len(nums):
        yield nums
        return
    for j in range(i, len(nums)): # iteración de n-i elementos.
        nums[i], nums[j] = nums[j], nums[i]
        yield from get_permutation(nums, i + 1)

# Ejemplo de permutación
p = get_permutation([1, 2, 3])
for x in p:
    print(x)

[1, 2, 3]
[1, 3, 2]
[3, 1, 2]
[3, 2, 1]
[1, 2, 3]
[1, 3, 2]


In [10]:
def fuerza_bruta(tomas, max_time=60):
    perm = get_permutation(tomas)
    start = time.time()
    end = time.time()
    best_sol = None
    best_cost = float("inf")
    i = 0
    for x in perm:
        sol = [x[MAX_TOMAS*(j-1):MAX_TOMAS*j] for j in range(1, 6)]
        cost = compute_cost(sol)
        if cost < best_cost:
            best_cost = cost
            best_sol = sol
        end = time.time()
        i += 1
        if end - start > max_time:
            break
    return best_sol, best_cost, i

secs = 600 # Diez minutos
sol, cost, iters = fuerza_bruta([i for i in range(NUM_TOMAS)], secs)
print(f"En {secs} segundos se han evaluado {iters} posibilidades de {TOTAL_POSIBILITIES}")
print("Esto supone un {} % de soluciones exploradas\n".format(iters / TOTAL_POSIBILITIES))

print_sol(sol, cost)

En 600 segundos se han evaluado 30904476 posibilidades de 265252859812191058636308480000000
Esto supone un 1.1650949219503806e-25 % de soluciones exploradas

La mejor solución es: 
[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]
[12, 13, 14, 15, 16, 17]
[18, 29, 27, 24, 22, 21]
[23, 28, 20, 25, 19, 26]
Tiene un coste de 33 con un total de 5 días de grabación


Calcula la complejidad del algoritmo por fuerza bruta

Obviamente el coste de la permutación es factorial, ya que se trata de una función recursiva. En la función hay un solo bucle  donde hemos anotado *n - i iteraciones*, y aquí añadimos que se realizan $0 < i < n$. Por tanto, lo que tenemos son n bucles anidados, dando las siguientes operaciones

$$
    \prod_{i=0}^{n}(n-i) = n!
$$

Vemos claro que el orden es $O(n!)$.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

(*)Calcula la complejidad del algoritmo

La función que veremos a continuación se trata de una función que realiza permutaciones también, pero esta vez teniendo en cuenta los subconjuntos creados, esto es, aplicando la restricción que hemos descrito más arriba.

In [287]:
from itertools import combinations

def permute_grouped(elements, k):
    t = len(elements) // k
    if t == 1:
        yield [elements]
        return
    first_item = next(iter(elements))
    rest_of_elements = elements - {first_item}
    comb = combinations(rest_of_elements, k - 1)
    for rest_of_group in comb: # comb son len(rest_of_elements) choose k-1
        rest_of_group = set(rest_of_group)
        group = {first_item} | rest_of_group
        for other_groups in permute_grouped(rest_of_elements - rest_of_group, k):
            yield [group] + other_groups

# Ejemplo
perm = permute_grouped({1, 2, 3, 4, 5, 6}, 2)
for p in perm:
    print(p)

[{1, 2}, {3, 4}, {5, 6}]
[{1, 2}, {3, 5}, {4, 6}]
[{1, 2}, {3, 6}, {4, 5}]
[{1, 3}, {2, 4}, {5, 6}]
[{1, 3}, {2, 5}, {4, 6}]
[{1, 3}, {2, 6}, {4, 5}]
[{1, 4}, {2, 3}, {5, 6}]
[{1, 4}, {2, 5}, {3, 6}]
[{1, 4}, {2, 6}, {3, 5}]
[{1, 5}, {2, 3}, {4, 6}]
[{1, 5}, {2, 4}, {3, 6}]
[{1, 5}, {2, 6}, {3, 4}]
[{1, 6}, {2, 3}, {4, 5}]
[{1, 6}, {2, 4}, {3, 5}]
[{1, 6}, {2, 5}, {3, 4}]


In [65]:
def fuerza_menos_bruta(tomas, max_time=60):
    perm = permute_grouped(set(tomas), MAX_TOMAS)
    start = time.time()
    best_sol = None
    best_cost = float("inf")
    i = 0
    for x in perm:
        sol = [list(el) for el in x]
        cost = compute_cost(sol)
        if cost < best_cost:
            best_cost = cost
            best_sol = sol
        end = time.time()
        i += 1
        if end - start > max_time:
            break
    return best_sol, best_cost, i

secs = 600 # Diez minutos
sol, cost, iters = fuerza_menos_bruta([i for i in range(NUM_TOMAS)], secs)
print(f"En {secs} segundos se han evaluado {iters} posibilidades de {EFFICIENT_POSIBILITIES}")
print("Esto supone un {} % de soluciones exploradas\n".format(iters / EFFICIENT_POSIBILITIES))

print_sol(sol, cost)

En 600 segundos se han evaluado 28052799 posibilidades de 11423951396577720
Esto supone un 2.455612600768224e-09 % de soluciones exploradas

La mejor solución es: 
[0, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 10, 11]
[28, 19, 25, 26, 12, 13]
[21, 24, 27, 29, 14, 15]
[16, 17, 18, 20, 22, 23]
Tiene un coste de 30 con un total de 5 días de grabación


Con la función *permute_grouped* la complejidad es la misma, ya que se trata de una función recursiva. Aún así, la profundidad de recursión es menor en esta función por lo que la complejidad no se dispara tanto como en la primera función.

La ventaja de este algoritmo es que cada solución que produce es nueva, no se repiten, por lo que ganamos el doble. Podemos ver que pese haber alcanzado un mediocre *2.45e-09 %*, con el primer método a fuerza bruta obteniamos un *7.36e-26 %* todavía más mediocre.

Además, comparando los dos métodos vemos que el segundo realiza más iteraciones, concretamente *28052799* frente a las *19523881* del primero. Por un lado esto se debe a lo que comentabamos antes de la profundidad de recursión menor, pero además, sin haberlo llegado a investigar en profundidad probablemente también se deba a que la implementación de *combinations* está totalmente optimizada, al igual que la resta entre objetos de tipo *set* en python y esto mejora sustancialmente nuestro algoritmo.

##### Problema 5:

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Nuestro algoritmo para encontrar soluciones aleatorias con sentido viene dado en la función *get_random_solution*, la cual genera soluciones a partir de una copia de las tomas a grabar. Se utiliza el set para operar entre conjuntos y evitar que se produzcan soluciones incorrectas.

In [304]:
import random

def get_random_solution():
    tomas = TOMAS_LIST.copy()
    sol = []
    while tomas: 
        day = random.sample(tomas, MAX_TOMAS)
        tomas = list(set(tomas) - set(day))
        sol.append(day)
    return sol
        

def random_solution(N=1e10, max_time=10):
    # Buscaremos soluciones random, la idea es todos los días hacer 6 tomas.
    best_sol = None
    best_cost = float("inf")
    start = time.time()
    end = time.time()
    i = 0
    while end - start < max_time and i < N:
        sol = get_random_solution()
        cost = compute_cost(sol)
        if cost < best_cost:
            best_cost = cost
            best_sol = sol
        end = time.time()
        i += 1
    return best_cost, best_sol

cost, sol = random_solution(max_time=10)
print_sol(sol, cost) # La mejor solución encontrada ha sido de 30
# get_random_solution()

La mejor solución es: 
[9, 26, 20, 1, 25, 5]
[18, 21, 10, 0, 6, 2]
[27, 3, 14, 24, 4, 15]
[11, 22, 8, 7, 13, 29]
[12, 16, 17, 23, 19, 28]
Tiene un coste de 30 con un total de 5 días de grabación


A modo de experimento, vamos a provar a generar soluciones que no necesariamente ocupen 6 días, sino que haya días en que se graben 4, 5 o 6 tomas, para ver si alguna de estas soluciones puede ser valida.

In [306]:
def more_random_solution(N=1e10, max_time=10):
    # Buscaremos soluciones random, esta vez cada día no se tiene porque hacer 6 tomas.
    # Los días tendrán entre 3 y 6 tomas.
    best_sol = None
    best_cost = float("inf")
    start = time.time()
    end = time.time()
    i = 0
    while end - start < max_time and i < N:
        sol = []
        tomas = [i for i in range(NUM_TOMAS)]
        while tomas:
            new_day = []
            upper_bound = min(len(tomas), MAX_TOMAS)
            lower_bound = 0 if upper_bound < 3 else 3
            num_tomas = random.randint(lower_bound, upper_bound)
            for i in range(num_tomas):
                toma = random.randint(0, len(tomas) - 1)
                new_day.append(tomas.pop(toma))
            sol.append(new_day)
        cost = compute_cost(sol)
        if cost < best_cost:
            best_cost = cost
            best_sol = sol
        i -= 1
        end = time.time()
    return best_cost, best_sol

cost, sol = more_random_solution(max_time=10)
print_sol(sol, cost) # La mejor solución encontrada ha sido de 33

La mejor solución es: 
[7, 25, 29, 9, 17, 18]
[28, 4, 3, 2, 24, 14]
[10, 13, 22, 21, 20, 6]
[5, 8, 0, 11, 23, 12]
[16, 27, 19, 26, 1, 15]
Tiene un coste de 31 con un total de 5 días de grabación


Conclusiones: Después de ejecutar varias veces cada algoritmo vemos que iterarlo 10000 veces 
apenas nos lleva 1~2 segundos.

La función *random_solution* nos ha brindado un mejor resultado, siendo
este de 30 en 5 días. 

La solución de *more_random_solution* también se ejecuta como una bala y los 
resultados son semejantes a los de *random_solution*, no obstante, todas las soluciones encontradas
son de 5 días, por lo que probablemente, todas las otras que contemplem más días están condenadas al fracaso. 

En conclusión, la segunda función se puede descartar ya que no vamos a encontrar una solución *milagrosa* buscando 
otras combinaciones de tomas que no sean seis diarias.

### Resolución eficiente del problema

Más que indicar las lineas de como vamos a resolver el problema, lo que haremos es programar los distintos métodos que se usan para solucionar problemas de este tipo.

#### Algoritmo voraz

Nuestra idea para construir un algoritmo voraz es la de que si agrupamos los actores en sus respectivas escenas, conseguiremos que el ese actor vaya los menos días posibles. Escogeremos un actor al azar, grabaremos sus escenas y después continuaremos por el actor que tiene al lado agrupando en la medida que se pueda las escenas que le toquen. Lo vemos en el siguiente ejemplo con cuatro acotres y 6 escenas. Antes pues, ejecutamos tres funciones que nos haran falta para crear este algoritmo voraz.

In [106]:
# Después de haber escogido el formato de datos con las filas de actores y las columnas de escenas,
# para este caso nos vendría bien que esté al revés. Básicamente lo que queremos es transponer la 
# matriz y para eso tenemos la siguiente función.

def transpose(tomas, save_row=True): # Orden O(n2)
    n = len(tomas)
    m = len(tomas[0])
    new_tomas = [[] for _ in range(m)]
    for i in range(n):
        for j in range(m):
            new_tomas[j].append(tomas[i][j])
    if save_row:
        for i, toma in enumerate(new_tomas):
            toma.append(i)
    return new_tomas

# Esta funcion permuta las columnas i y j de una matriz.
def swap_order(tomas, i, j): # O(1)
    tomas[i], tomas[j] = tomas[j], tomas[i]
    return tomas

# Este algoritmo ordena las listas por cada uno de sus items
def greedy_choose(tomas, level=0, size=NUM_ACTORS-1): # O(nm)
    if level == size:
        return tomas
    result = []
    while tomas:
        pos = -1
        for i, val in enumerate(tomas):
            order = val[level]
            if order > pos:
                pos = order
                conflicts = []
            if order == pos:
                conflicts.append(val)
        if len(conflicts) == 1:
            result += conflicts
        else:
            result += greedy_choose(conflicts.copy(), level+1)
        for con in conflicts:
            tomas.remove(con)
    return result

In [308]:
# Ejemplo
T = [[1, 1, 0, 1], [1, 0, 1, 1], [1, 0, 0, 1], [0, 1, 1, 1], [0, 1, 0, 1], [0, 0, 1, 1]]
sol = greedy_choose(T.copy(), size=3)
print(*sol, sep="\n")

[1, 1, 0, 1]
[1, 0, 1, 1]
[1, 0, 0, 1]
[0, 1, 1, 1]
[0, 1, 0, 1]
[0, 0, 1, 1]


Podemos ver que no es la mejor de las soluciones y además, que esta depende de el orden en el que escogemos los actores, por ejemplo, si intercambiamos la fila 1 por la 4, la solución cambiaría. 

In [309]:
T = transpose(T.copy(), save_row=False)
T = swap_order(T.copy(), 0, 3)
T = transpose(T.copy(), save_row=False)
sol = greedy_choose(T.copy(), size=3)
print(*sol, sep="\n")

[1, 1, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 0]
[1, 0, 1, 1]
[1, 0, 1, 0]
[1, 0, 0, 1]


Vemos entonces, que nuestro problema voraz es realmente, en este punto un problema de ordenación. Sabiendo esto, lo que vamos a hacer es explorar todos los distintos tipos de ordenaciones que podemos obtener realizando todos los distintos swaps entre las columnas.

In [311]:
def greedy_alg(): # Tenemos NUM_ACTORS x NUM_ACTORS * (n * m + n + n^2) operaciones aproximadamente en la ejecución
    best_sol = None
    best_cost = float("inf")
    for i in range(NUM_ACTORS-2):
        for j in range(i, NUM_ACTORS-1):
            tomas = transpose(swap_order(TOMAS.copy(), i, j))
            tomas = greedy_choose(tomas)
            sol = []
            day = []
            for k, toma in enumerate(tomas):
                day.append(toma[-1])
                if not (k + 1) % MAX_TOMAS:
                    sol.append(day.copy())
                    day = []
            cost = compute_cost(sol)
            if cost < best_cost:
                best_cost = cost
                best_sol = sol
    return best_cost, best_sol

cost, sol = greedy_alg()
print_sol(sol, cost)

La mejor solución es: 
[10, 4, 3, 20, 0, 11]
[21, 5, 6, 8, 24, 2]
[9, 7, 14, 19, 1, 25]
[13, 17, 23, 16, 18, 22]
[12, 26, 27, 29, 15, 28]
Tiene un coste de 29 con un total de 5 días de grabación


Por primera vez hemos conseguido bajar de 30 en nuestra función de coste!

Nuestro objetivo ahora será encontrar una función que haga una búsqueda local para mejorar nuestras soluciones. PAra ello, presentamos la siguiente función encargada de intercambiar dos tomas de dos días distintos.

In [317]:
def swap_tomas(sol, l1, l2): # m x m operaciones, con m = MAX_TOMAS
    # sol <list>: solucion standard
    # l1, l2 <int>: los días sobre los que se permuta
    best_cost = compute_cost(sol)
    best_sol = sol
    m = len(sol[0])
    for i in range(m):
        for j in range(m):
            sol[l1][i], sol[l2][j] = sol[l2][j], sol[l1][i]
            cost = compute_cost(sol)
            if cost < best_cost:
                best_cost = cost
                best_sol = [s.copy() for s in sol]
            sol[l1][i], sol[l2][j] = sol[l2][j], sol[l1][i]
    return best_sol, best_cost

sol = [[i for i in range(MAX_TOMAS*(j-1), MAX_TOMAS*j)] for j in range(1, MAX_TOMAS)]
best_sol, best_cost = swap_tomas(sol.copy(), 0, 1)
print_sol(best_sol, best_cost)

La mejor solución es: 
[10, 1, 2, 3, 4, 5]
[6, 7, 8, 9, 0, 11]
[12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29]
Tiene un coste de 35 con un total de 5 días de grabación


In [318]:
# Función que dada una solución encuentra su mejor solución local.

def busqueda_local(sol): # (m / n)^2 x MAX_TOMAS^2
    days = len(sol)
    tomas = len(sol[0])
    best_cost = compute_cost(sol)
    best_sol = sol
    improve = False
    for i in range(days-1):
        for j in range(i, days):
            sol, cost = swap_tomas(sol, i, j)
            if cost < best_cost:
                improve = True
                best_sol = sol
                best_cost = cost
    if improve:
        best_sol, best_cost = busqueda_local(best_sol)
    return best_sol, best_cost

sol = [[i for i in range(MAX_TOMAS*(j-1), MAX_TOMAS*j)] for j in range(1, MAX_TOMAS)]

best_sol, best_cost = busqueda_local(sol)
print_sol(best_sol, best_cost)

La mejor solución es: 
[10, 14, 2, 3, 24, 5]
[6, 7, 8, 9, 0, 11]
[28, 13, 1, 15, 16, 17]
[18, 4, 20, 21, 22, 23]
[19, 25, 26, 27, 12, 29]
Tiene un coste de 30 con un total de 5 días de grabación


Finalmente vamos a aplicar el algoritmo voraz con busqueda local.

In [319]:
# Función que busca las mejores soluciones voraces y le realiza una busqueda local a cada una de ellas.

def greedy_plus_local_search():
    best_sol = None
    best_cost = float("inf")
    for i in range(NUM_ACTORS-2):
        for j in range(i, NUM_ACTORS-1):
            tomas = transpose(swap_order(TOMAS.copy(), i, j))
            tomas = greedy_choose(tomas)
            sol = []
            day = []
            for k, toma in enumerate(tomas):
                day.append(toma[-1])
                if not (k + 1) % MAX_TOMAS:
                    sol.append(day.copy())
                    day = []
            sol, cost = busqueda_local(sol)
            if cost < best_cost:
                best_cost = cost
                best_sol = sol
    return best_cost, best_sol

cost, sol = greedy_plus_local_search()
print_sol(sol, cost)

La mejor solución es: 
[0, 1, 21, 10, 19, 5]
[13, 16, 18, 22, 23, 17]
[11, 25, 7, 8, 24, 9]
[6, 3, 14, 4, 2, 12]
[27, 29, 28, 26, 15, 20]
Tiene un coste de 27 con un total de 5 días de grabación


Esto va mejorando, con este metodo hemos reducido dos puntos la solución en un tiempo de ejecución mínimo. Nuestro objetibo ahora será añadir una parte de aleatoriedad en el algoritmo voraz y de esta manera poder generar soluciones distintas para tenerlo buscando respuestas durante 3, 4 o 5 minutos.

#### GRASP

EL metaheurístico GRASP (greedy randomized adaptive search procedure) nos puede proporcionar una mejor solución ya que nos permite a través de la aleatoriedad explorar más soluciones que en el método voraz se descartan pudiendo ser buenas soluciones.

In [320]:
# Este algoritmo ordena las listas por cada uno de sus items. El resultado es el mismo que
# el de greedy_choose con la diferencia que este es generativo y nos devuelve los elementos
# ordenados uno a uno.

def greedy_choose_gen(tomas, level=0, size=NUM_ACTORS-1):
    if level == size:
        yield tomas
        return
    result = []
    while tomas:
        pos = -1
        for i, val in enumerate(tomas):
            order = val[level]
            if order > pos:
                pos = order
                conflicts = []
            if order == pos:
                conflicts.append(val)
        if len(conflicts) == 1:
            result += conflicts
        else:
            result += greedy_choose(conflicts.copy(), level+1)
        for con in conflicts:
            tomas.remove(con)
    yield from result

In [322]:
# Aquí hay un pequeño bug que no he podido resolver, y es que bajo ciertos parametros
# la función devuelve una lista vacía. Esto afecta a nuestro caso por lo que tendremos
# que descartar esto. Por lo demás funciona correctamente.


tomas = transpose(TOMAS.copy())
sol = greedy_choose_gen(tomas)
for g in sol:
    if not g: # Descartar lista vacía
        continue
    print(g)

[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 1, 0, 0, 0, 11]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 21]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 10]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 5]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 6]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 8]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 24]
[1, 1, 0, 0, 0, 1, 0, 0, 1, 9]
[1, 1, 0, 0, 0, 1, 0, 0, 0, 7]
[1, 1, 0, 0, 0, 0, 1, 1, 0, 3]
[1, 1, 0, 0, 0, 0, 1, 0, 0, 14]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 19]
[1, 0, 1, 0, 1, 0, 0, 0, 1, 25]
[1, 0, 1, 0, 0, 1, 0, 0, 0, 13]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 16]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 18]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 22]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 12]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 27]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 29]
[1, 0, 0, 0, 1, 1, 0, 0, 0, 28]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 4]
[0, 1, 0, 0, 1, 0, 1, 0, 0, 2]
[0, 0, 1, 1, 1, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 1, 0, 0, 0, 17]
[0, 0, 1, 0, 0, 1, 0, 0, 0, 23]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 26]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 15]
[0, 0, 0, 0, 0, 1, 0, 1, 0, 20]


En este punto ya podemos definir nuestro algoritmo voraz con factor aleatorio. Este factor lo implementaremos generando $6<n<12$ elementos ordenados y mezclandolos para de esta manera alterar la solución netamente ordenada.

In [258]:
import random

def greedy_randomized_alg(rand_fact=9):
    best_sol = None
    best_cost = float("inf")
    for i in range(NUM_ACTORS-2):
        for j in range(i, NUM_ACTORS-1):
            tomas = transpose(swap_order(TOMAS.copy(), i, j))
            gen = greedy_choose_gen(tomas)
            tomas = []
            batch = []
            for k, g in enumerate(gen):
                if not g:
                    continue
                batch.append(g)
                if k == rand_fact: # Una vez hay rand_fact elementos ordenados los mezclamos.
                    random.shuffle(batch)
                    tomas += batch # Añadimos los elementos mezclados a la solución
                    batch = []
            random.shuffle(batch)
            tomas += batch
            sol = []
            day = []
            for k, toma in enumerate(tomas):
                day.append(toma[-1])
                if not (k + 1) % MAX_TOMAS:
                    sol.append(day.copy())
                    day = []
            cost = compute_cost(sol)
            if cost < best_cost:
                best_cost = cost
                best_sol = [s.copy() for s in sol]
    return best_cost, best_sol

cost, sol = greedy_randomized_alg()
print_sol(sol, cost)

La mejor solución es: 
[8, 5, 0, 6, 4, 10]
[11, 21, 20, 3, 24, 2]
[9, 7, 14, 19, 1, 25]
[13, 17, 23, 16, 18, 22]
[12, 26, 27, 29, 15, 28]
Tiene un coste de 29 con un total de 5 días de grabación


In [326]:
def grasp(max_time=60):
    best_sol = None
    best_cost = float("inf")
    st = time.time()
    et = time.time()
    while et - st < max_time:
        rand_fact = random.randint(7, 11)
        for i in range(NUM_ACTORS-2):
            for j in range(i, NUM_ACTORS-1):
                tomas = transpose(swap_order(TOMAS.copy(), i, j))
                cost, sol = greedy_randomized_alg(11)
                sol, cost = busqueda_local(sol)
                if cost == best_cost:
                    print("Solución alternativa!")
                if cost < best_cost:
                    print(
                        "Nueva solución encontrada! {} segundos de ejecucion".format(
                            time.time()-st
                        )
                    )
                    best_cost = cost
                    best_sol = [s.copy() for s in sol]
        et = time.time()
    return best_cost, best_sol

cost, sol = grasp(10)
print_sol(sol, cost)

Nueva solución encontrada! 0.02428460121154785 segundos de ejecucion
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Nueva solución encontrada! 0.19585108757019043 segundos de ejecucion
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
Solución alternativa!
La mejor solución es: 
[3, 20, 14, 7, 2, 28]
[24, 5, 10, 4, 8, 0]
[9, 11, 21, 19, 1, 25]
[13, 17, 23, 16, 18, 22]
[12, 26, 27, 29, 15, 6]
Tiene un coste de 26 con un total de 5 días de grabación


Vemos que la función deja de mejorar al poco de comenzar y como mucho encuentra soluciones equivalentes, pero no mejores.

#### Conclusiones

Después de aplicar todos los métodos, hemos visto como la solución mejora considerablemente a medida que vamos aplicando técnicas productivas y poco costosas. Hemos pasado de soluciones malas de 31/32 a 26 con el último método.

Nunca sabremos si este es el óptimo, pero sacando calculos rápidos, vemos que hay un actor que mínimo ha de ir 4 días, tres que mínimo 3, dos que mínimo 2 y cuatro que les basta un día. Sumando esto, tenemos $4 + 3\times 3 + 2\times 2 + 4 \times 1 = 21$, esto teniendo en cuenta que se puede combinar a la perfección, cosa que no ocurre porque hubiesemos encontrado esa solución mediante el algoritmo voraz. En resumen, puede que no sea la solución optima, pero es una buena solución.

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

La bibiliografía usada en el trabajo ha sido la siguiente:

    - https://dmle.icmat.es/pdf/GACETAMATEMATICA_1974_26_3-4_02.pdf
    - https://stackoverflow.com/questions/53419536/what-is-the-computational-complexity-of-itertools-combinations-in-python
